# Metrics and Features
This notebook shows examples of using metrics and features together in V8. It covers:
1. Retrieving metric data using the new `eval()` API.
2. Creating features from metrics, creating feature sets referencing such features, and publishing the features / feature sets.
3. Defining features and feature sets directly as seed data and loading them into the application.

### Metrics, Features, and Feature Sets used
Here are the features and metrics used in this notebook:

| Field                  | Metric                     | How created | Feature                           | How created |
| ---------------------- | -------------------------- | ----------- | --------------------------------- | ----------- |
| activePower            | ActivePowerAvg             | In feature  | activePowerAvgFeature             | API         |
| generatorRotationSpeed | GeneratorRotationSpeedAvg  | Metadata    | generatorRotationSpeedAvgFeature  | API         |
| gearOilTemperature     | GearOilTemperatureAvg      | Metadata    | gearOilTemperatureAvgFeature      | API         |
| activePower            | ActivePowerDiff            | In feature  | activePowerDiffFeature            | Seed        |
| generatorRotationSpeed | GeneratorRotationSpeedDiff | Metadata    | generatorRotationSpeedDiffFeature | Seed        |
| gearOilTemperature     | GearOilTemperatureDiff     | Metadata    | gearOilTemperatureDiffFeature     | Seed        |
| eventCode              | HasFailureEvent            | Metadata    | N/A                               | N/A         |
| N/A                    | WillFailNextDay            | Matadata    | willFailNextDayFeature            | Seed        |

Here are the feature sets used in this notebook:

| Feature Set        | How created | Features                                                                                 |
| ------------------ | ----------- | ---------------------------------------------------------------------------------------- |
| metricsFeatureSet1 | API         | activePowerAvgFeature, generatorRotationSpeedAvgFeature, gearOilTemperatureAvgFeature    |
| metricsFeatureSet2 | Seed        | activePowerDiffFeature, generatorRotationSpeedDiffFeature, gearOilTemperatureDiffFeature |
| labelFeatureSet    | Seed        | willFailNextDayFeature                                                                   |     


## Prerequistes
1. To run this notebook, you must first create an app using `windTurbineWorkshop` as the root package.
2. You should run the `1-Generate-Demo-Data.ipynb` to populate the Wind Turbine tables.

## Setup

In [1]:
# clean up metrics that will be created in this tutorial
c3.Feature.removeAll(filter="id=='WindTurbine#activePowerAvgFeature'", confirm=True)
c3.Feature.removeAll(filter="id=='WindTurbine#generatorRotationSpeedAvgFeature'", confirm=True)
c3.Feature.removeAll(filter="id=='WindTurbine#gearOilTemperatureAvgFeature'", confirm=True)
c3.Feature.Set.removeAll(filter="id=='WindTurbine#metricsFeatureSet1'", confirm=True)

1

Filter we can use to select a specific turbine's data

In [2]:
SUBJECT_FILTER="name=='TURBINE-1'"

In [3]:
assert len(c3.WindTurbine.eval())>0, "You need to run the 1-Generate-Demo-Data.ipynb notebook first!"

## Review the loaded data
We first take a look at the three entities we will be using -- `WindTurbine`, `WindTurbineMeasurementSeries`, and `WindTurbineMeasurement`.

In [4]:
c3.WindTurbine.eval()

id       name location  power  manufacturer
0  demo_TURBINE-1  TURBINE-1      USA    150  demo_Siemens
1  demo_TURBINE-2  TURBINE-2      USA    200  demo_Siemens
2  demo_TURBINE-3  TURBINE-3   FRANCE    150  demo_Siemens
3  demo_TURBINE-4  TURBINE-4      USA    200       demo_GE
4  demo_TURBINE-5  TURBINE-5   FRANCE    150       demo_GE
5  demo_TURBINE-6  TURBINE-6      USA    150  demo_Siemens

In [5]:
c3.WindTurbineMeasurementSeries.eval()

earliest              latest  ...     windTurbine treatment
0 2022-01-01 00:00:02 2022-06-01 00:44:47  ...  demo_TURBINE-1      rate
1 2022-01-01 00:58:05 2022-06-01 00:36:20  ...  demo_TURBINE-2      rate
2 2022-01-01 00:15:45 2022-05-31 19:49:25  ...  demo_TURBINE-3      rate
3 2022-01-01 00:30:09 2022-06-01 00:17:41  ...  demo_TURBINE-4      rate
4 2022-01-01 01:19:39 2022-05-31 21:05:42  ...  demo_TURBINE-5      rate
5 2022-01-01 00:21:36 2022-06-01 00:46:22  ...  demo_TURBINE-6      rate

[6 rows x 5 columns]

In [6]:
c3.WindTurbineMeasurement.eval().head()

start  isEstimated  ... generatorRotationSpeed activePower
0 2022-01-01 00:00:02        False  ...                 1330.0      5109.0
1 2022-01-01 00:09:12        False  ...                 1366.0      4828.0
2 2022-01-01 01:58:39        False  ...                 1369.0      4845.0
3 2022-01-01 03:11:51        False  ...                 1318.0      4762.0
4 2022-01-01 05:52:41        False  ...                 1380.0      5362.0

[5 rows x 7 columns]

## 1. Use the `eval` call to retrieve metric data
Given a `FeatureEvaluatable` type like `WindTurbine`, we can use the `eval()` method to retrieve fields, metrics, and features.
Here, we show how it can be used to retrieve metric data.

We can dynamically create a metric definition to evaluate by placing the definition in `overrideMetrics`
parameter of `eval()`. We can then refer to the metric by name in the `projection` parameter.

In [7]:
metric_1 = c3.SimpleMetric.make({
    'id': 'WindTurbine_ActivePowerAvg',
    'name': 'ActivePowerAvg',
    'srcType': 'WindTurbine',
    'path': 'turbineMeasurements',
    'expression': "window('AVG', avg(normalized.data.activePower), -23, 24, 1)"
})

eval_result = c3.WindTurbine.eval(filter=SUBJECT_FILTER, projection='ActivePowerAvg',
                                  interval='HOUR', start='2022-01-02', end='2022-01-03',
                                  overrideMetrics=[metric_1])

eval_result.head()

subject           timestamp  ActivePowerAvg
0  demo_TURBINE-1 2022-01-02 00:00:00     4971.270833
1  demo_TURBINE-1 2022-01-02 01:00:00     4990.458333
2  demo_TURBINE-1 2022-01-02 02:00:00     5019.041667
3  demo_TURBINE-1 2022-01-02 03:00:00     5049.354167
4  demo_TURBINE-1 2022-01-02 04:00:00     5081.395833

We next query for two metrics that we have included in the package metadata. Since the metrics are already
defined, we just refer to them by name in the `projection` parameter.

In [8]:
eval_result = c3.WindTurbine.eval(projection='GeneratorRotationSpeedAvg, GearOilTemperatureAvg', 
                                  filter=SUBJECT_FILTER, interval='HOUR', start='2022-01-02', 
                                  end='2022-01-03')
eval_result.head()

subject  ... GearOilTemperatureAvg
0  demo_TURBINE-1  ...             61.364583
1  demo_TURBINE-1  ...             60.558333
2  demo_TURBINE-1  ...             59.766667
3  demo_TURBINE-1  ...             59.370833
4  demo_TURBINE-1  ...             59.370833

[5 rows x 4 columns]

## 2. Creating features from metrics using APIs

Now, let's create some features on top of metrics. Although we were able to directly
`eval()` the metrics, defining a `Feature` will enable caching of the data in the feature store and make it available
to use as an input to `MlModels`.


We can use `Feature.fromMetric()` and reference either the metric name or provide a definition dynamically. We then call `create()` to save to the database.



### Creating a feature from a dynamically defined metric
For our first feature, we dynamically define a metric `ActivePowerAvg` in the `metric` field of `LegacyMetric`.

In [9]:
ActivePowerAvg = c3.SimpleMetric.make({
    'id': 'WindTurbine_ActivePowerAvg',
    'name': 'ActivePowerAvg',
    'srcType': 'WindTurbine',
    'path': 'turbineMeasurements',
    'expression': "window('AVG', avg(normalized.data.activePower), -23, 24, 1)"
})
ActivePowerAvg

{
  "type" : "SimpleMetric",
  "name" : "ActivePowerAvg",
  "expression" : "window('AVG', avg(normalized.data.activePower), -23, 24, 1)",
  "id" : "WindTurbine_ActivePowerAvg",
  "srcType" : "WindTurbine",
  "path" : "turbineMeasurements"
}

In [10]:
f1 = c3.Feature.fromMetric(subjectType=c3.WindTurbine,
                           legacy=c3.LegacyMetric(metric=ActivePowerAvg, interval='HOUR'),
                           name="activePowerAvgFeature")
f1 = f1.withDescription("Rolling average over 24 hours, created via API")
f1 = f1.create()
f1 = f1.get()
f1

{
  "type" : "Feature",
  "id" : "WindTurbine#activePowerAvgFeature",
  "name" : "activePowerAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling average over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "ActivePowerAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "overrideMetrics" : [ {
              "type" : "SimpleMetric",


We can use the method `toPySrc()` on the feature definition to see the equivalent python code to generate the feature.

In [11]:
print(f1.toPySrc())

tmp_df0 = WindTurbine.eval(projection='ActivePowerAvg', interval='HOUR', timeZone={
  "type" : "TimeZone",
  "name" : "NONE"
}, overrideMetrics=[{
  "type" : "SimpleMetric",
  "name" : "ActivePowerAvg",
  "expression" : "window('AVG', avg(normalized.data.activePower), -23, 24, 1)",
  "id" : "WindTurbine_ActivePowerAvg",
  "srcType" : "WindTurbine",
  "path" : "turbineMeasurements"
}], materialize=True)
ret = tmp_df0


### Creating features from a metric defined via metadata
For the next two features, we will reference by name metrics defined in the application metadata.

In [12]:
f2 = c3.Feature.fromMetric(subjectType=c3.WindTurbine, 
                           legacy=c3.LegacyMetric(metric="GeneratorRotationSpeedAvg", interval='HOUR'), 
                           name="generatorRotationSpeedAvgFeature")

f2 = f2.withDescription("Rolling avererage over 24 hours, created via API")
f2 = f2.create()
f2 = f2.get()
f2

{
  "type" : "Feature",
  "id" : "WindTurbine#generatorRotationSpeedAvgFeature",
  "name" : "generatorRotationSpeedAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling avererage over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "GeneratorRotationSpeedAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "materialize" : true
       

We can view the source code of this metric as well. Note that it does *not* include the definition of the metric.

In [13]:
print(f2.toPySrc())

tmp_df0 = WindTurbine.eval(projection='GeneratorRotationSpeedAvg', interval='HOUR', timeZone={
  "type" : "TimeZone",
  "name" : "NONE"
}, materialize=True)
ret = tmp_df0


In [14]:
f3 = c3.Feature.fromMetric(subjectType=c3.WindTurbine, 
                           legacy=c3.LegacyMetric(metric="GearOilTemperatureAvg", interval='HOUR'), 
                           name="gearOilTemperatureAvgFeature")

f3 = f3.withDescription("Rolling avererage over 24 hours, created via API")
f3 = f3.create()
f3 = f3.get()
f3

{
  "type" : "Feature",
  "id" : "WindTurbine#gearOilTemperatureAvgFeature",
  "name" : "gearOilTemperatureAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling avererage over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "GearOilTemperatureAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "materialize" : true
          }
       

### Materialize and eval the features
Before we can read our feature data, we must `materialize` the features first. In production, this will be handled by a cron job.

---
**Note:**

Materialization for metric-based features goes back only 5 years by default. This is because metrics require a time range to evaluate.
If you have data that goes back more than five years, you can address this by setting `materializeTimeRange` on each feature when you create it.
For example, when creating `activePowerAvgFeature`, we can set the feature to only materialize data from the year 2012 as follows:
```
f1 = c3.Feature.fromMetric(subjectType=c3.WindTurbine,
                           legacy=c3.LegacyMetric(metric=ActivePowerAvg, interval='HOUR'),
                           name="activePowerAvgFeature")
f1 = f1.withMaterializeTimeRange(c3.Pair.ofStr('dateTime("2012-01-01")', 'dateTime("2013-01-01")'),)
f1 = f1.withDescription("Rolling avererage over 24 hours, created via API")
f1 = f1.create()
```

---



In [15]:
f1.materialize(sync=True)
f2.materialize(sync=True)
f3.materialize(sync=True)

We can now use `eval()` to retrieve our features.

In [16]:
projection = 'activePowerAvgFeature, generatorRotationSpeedAvgFeature, gearOilTemperatureAvgFeature'
eval_result = c3.WindTurbine.eval(projection=projection, filter=SUBJECT_FILTER, 
                                  start='2022-01-02', end='2022-01-03', interval='HOUR')

eval_result.head()

subject  ... gearOilTemperatureAvgFeature
0  demo_TURBINE-1  ...                    61.364583
1  demo_TURBINE-1  ...                    60.558333
2  demo_TURBINE-1  ...                    59.766667
3  demo_TURBINE-1  ...                    59.370833
4  demo_TURBINE-1  ...                    59.370833

[5 rows x 5 columns]

We can also use feature-specific APIs, like `evalFeaturesBatch()`

In [17]:
feature_list = ['activePowerAvgFeature', 'generatorRotationSpeedAvgFeature', 'gearOilTemperatureAvgFeature']
eval_result = c3.WindTurbine.evalFeaturesBatch(features=feature_list, filter=SUBJECT_FILTER,
                                               start='2022-01-02', end='2022-01-03')
eval_result.head()

subject  ... gearOilTemperatureAvgFeature
0  demo_TURBINE-1  ...                    61.364583
1  demo_TURBINE-1  ...                    60.558333
2  demo_TURBINE-1  ...                    59.766667
3  demo_TURBINE-1  ...                    59.370833
4  demo_TURBINE-1  ...                    59.370833

[5 rows x 5 columns]

### Create a feature set
Now, we can build a feature set from these features.

In [18]:
feature_set = c3.Feature.Set(name='metricsFeatureSet1',
                             id='WindTurbine#metricsFeatureSet1',
                             subjectType=c3.WindTurbine,
                             features=['activePowerAvgFeature',
                                       'generatorRotationSpeedAvgFeature',
                                       'gearOilTemperatureAvgFeature'])
feature_set = feature_set.withInterval("HOUR")  # a feature set has an associated interval to be evaluated
feature_set = feature_set.withDescription("Rolling average features, created via API")
feature_set = feature_set.create()
feature_set = feature_set.get() # need to call get() to obtain full definition
feature_set

{
  "type" : "Feature.Set",
  "id" : "WindTurbine#metricsFeatureSet1",
  "name" : "metricsFeatureSet1",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:25Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:25Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:25Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature.Set"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling average features, created via API",
  "interval" : "HOUR",
  "features" : [ "generatorRotationSpeedAvgFeature", "activePowerAvgFeature", "gearOilTemperatureAvgFeature" ]
}

If we try to retrieve the feature set data, we don't find anything yet.

In [19]:
eval_fs_result = c3.WindTurbine.evalFeatureSetBatch(filter=SUBJECT_FILTER, featureSet=feature_set, 
                                                    start='2022-01-02', end='2022-01-03')

eval_fs_result.head()

subject  ... generatorRotationSpeedAvgFeature
0  demo_TURBINE-1  ...                      1363.770833
1  demo_TURBINE-1  ...                      1359.666667
2  demo_TURBINE-1  ...                      1356.625000
3  demo_TURBINE-1  ...                      1354.645833
4  demo_TURBINE-1  ...                      1353.729167

[5 rows x 5 columns]

We need to explicitly materialize the feature set.

In [20]:
job = feature_set.materialize(sync=False)
job.waitForCompletion()

Now, we can retrieve data from this feature set using `evalFeatureSetBatch`.

In [21]:
eval_fs_result = c3.WindTurbine.evalFeatureSetBatch(filter=SUBJECT_FILTER, featureSet=feature_set, 
                                                    start='2022-01-02', end='2022-01-03')

eval_fs_result.head()

subject  ... generatorRotationSpeedAvgFeature
0  demo_TURBINE-1  ...                      1363.770833
1  demo_TURBINE-1  ...                      1359.666667
2  demo_TURBINE-1  ...                      1356.625000
3  demo_TURBINE-1  ...                      1354.645833
4  demo_TURBINE-1  ...                      1353.729167

[5 rows x 5 columns]

### Publish features and feature sets as seed data

We can now call the `publish()` method on each feature and the feature set to have them written as seed data to the package.

In [22]:
c3.Pkg.setDevMode(True) # we need to be in dev mode to write seed data
f1.publish()

{
  "type" : "Feature",
  "id" : "WindTurbine#activePowerAvgFeature",
  "name" : "activePowerAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling average over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "ActivePowerAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "overrideMetrics" : [ {
              "type" : "SimpleMetric",


The published feature has been written under the package's root directory at `seed/Feature/WindTurbine#activePowerAvgFeature.json`

In [23]:
f2.publish()

{
  "type" : "Feature",
  "id" : "WindTurbine#generatorRotationSpeedAvgFeature",
  "name" : "generatorRotationSpeedAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling avererage over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "GeneratorRotationSpeedAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "materialize" : true
       

In [24]:
f3.publish()

{
  "type" : "Feature",
  "id" : "WindTurbine#gearOilTemperatureAvgFeature",
  "name" : "gearOilTemperatureAvgFeature",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:21Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:21Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:21Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling avererage over 24 hours, created via API",
  "_data" : {
    "type" : "Data.Lazy",
    "lazies" : {
      "0" : {
        "type" : "Data.Lazy",
        "this" : "WindTurbine",
        "action" : "eval",
        "args" : {
          "spec" : {
            "type" : "EvalSpec",
            "projection" : "GearOilTemperatureAvg",
            "interval" : "HOUR",
            "timeZone" : {
              "name" : "NONE"
            },
            "materialize" : true
          }
       

We can also publish the feature set. This will be written under the package at `seed/Feature.Set/WindTurbine#metricsFeatureSet1.json`.

In [25]:
feature_set.publish()

{
  "type" : "Feature.Set",
  "id" : "WindTurbine#metricsFeatureSet1",
  "name" : "metricsFeatureSet1",
  "meta" : {
    "appCode" : 1765441463617436794,
    "env" : "c3",
    "app" : "wt45",
    "created" : "2023-05-09T18:42:25Z",
    "createdBy" : "BA",
    "updated" : "2023-05-09T18:42:25Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-05-09T18:42:25Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature.Set"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "description" : "Rolling average features, created via API",
  "interval" : "HOUR",
  "features" : [ "generatorRotationSpeedAvgFeature", "activePowerAvgFeature", "gearOilTemperatureAvgFeature" ]
}

### Reviewing our work so far

Now, let's look at what we've created.

In [26]:
c3.Feature.eval(projection='id,name,description')

subject  ...                                        description
0                   WindTurbine#activePowerAvgFeature  ...     Rolling average over 24 hours, created via API
1                  WindTurbine#activePowerDiffFeature  ...      Metric-backed feature, seeded in application.
2                     WindTurbine#activePower_diff_ml  ...                                               None
3         WindTurbine#activePower_median_deviation_ml  ...                                               None
4                          WindTurbine#activePower_ml  ...                                               None
5             WindTurbine#activePower_rolling_mean_ml  ...                                               None
6              WindTurbine#activePower_rolling_std_ml  ...                                               None
7            WindTurbine#gearOilTemperatureAvgFeature  ...   Rolling avererage over 24 hours, created via API
8           WindTurbine#gearOilTemperatureDiffFeature  ...  Metric-backed feature, seeded in application. ...
9              WindTurbine#gearOilTemperature_diff_ml  ...                                               None
10  WindTurbine#gearOilTemperature_median_deviatio...  ...                                               None
11                  WindTurbine#gearOilTemperature_ml  ...                                               None
12     WindTurbine#gearOilTemperature_rolling_mean_ml  ...                                               None
13      WindTurbine#gearOilTemperature_rolling_std_ml  ...                                               None
14       WindTurbine#generatorRotationSpeedAvgFeature  ...   Rolling avererage over 24 hours, created via API
15      WindTurbine#generatorRotationSpeedDiffFeature  ...  Metric-backed feature, seeded in application. ...
16         WindTurbine#generatorRotationSpeed_diff_ml  ...                                               None
17  WindTurbine#generatorRotationSpeed_median_devi...  ...                                               None
18              WindTurbine#generatorRotationSpeed_ml  ...                                               None
19  WindTurbine#generatorRotationSpeed_rolling_mea...  ...                                               None
20  WindTurbine#generatorRotationSpeed_rolling_std_ml  ...                                               None
21                 WindTurbine#willFailNextDayFeature  ...  A seeded feature that can be used as a label. ...

[22 rows x 3 columns]

In [27]:
c3.Feature.Set.eval(projection='id,name,description')

subject  ...                                        description
0         WindTurbine#labelFeatureSet  ...  Feature is True if a failure should be predict...
1      WindTurbine#metricsFeatureSet1  ...          Rolling average features, created via API
2      WindTurbine#metricsFeatureSet2  ...  These are metrics-backed features, created via...
3  WindTurbine#windTurbineFeatures_ml  ...  \nThis is a time series feature set with featu...

[4 rows x 3 columns]

## 3. Defining features and feature sets as seed

We have seeded several features and one feature set as seed data. We will materialize the features and retrieve data via `eval`.

### Seed data format
Here is an example of a feature with the metric defined inline. In the file
`seed/Feature/WindTurbine#actviePowerDiffFeature.json`, we have:
```
{
  "type": "Feature",
  "id": "WindTurbine#activePowerDiffFeature",
  "name": "activePowerDiffFeature",
  "subjectType": "WindTurbine",
  "legacy": {
    "metric": {
      "type": "SimpleMetric",
      "id": "WindTurbine_ActivePowerDiff",
      "name": "ActivePowerDiff",
      "srcType": "WindTurbine",
      "path": "turbineMeasurements",
      "expression": "window(\"MAX\", rollingDiff(avg(normalized.data.activePower)), -23, 24, 1)"
    },
    "interval": "HOUR"
  },
  "description": "max difference, seeded in application"
}
```

Here is an example of a feature referencing a metric defined in metadata and referenced by
name. In the file `seed/Feature/WindTurbine#gearOilTemperatureDiffFeature.json`, we have:
```
{
  "type": "Feature",
  "id": "WindTurbine#gearOilTemperatureDiffFeature",
  "name": "gearOilTemperatureDiffFeature",
  "subjectType": "WindTurbine",
  "legacy": {
    "metric": "GearOilTemperatureDiff",
    "interval": "HOUR"
  },
  "description": "max difference, seeded in application"
}
```

Here is another feature referencing a metric, in the file
`seed/Feature/WindTurbine#generatorRotationSpeedDiffFeature.json`:
```
{
  "type": "Feature",
  "id": "WindTurbine#generatorRotationSpeedDiffFeature",
  "name": "generatorRotationSpeedDiffFeature",
  "subjectType": "WindTurbine",
  "legacy": {
    "metric": "GeneratorRotationSpeedDiff",
    "interval": "HOUR"
  },
  "description": "max difference, seeded in application"
}
```

Here is a feature referencing a *compound metric* by name, from the file
`seed/Feature/WindTurbine#WillFailNextDayFeature.json`:
```
{
  "type": "Feature",
  "id": "WindTurbine#willFailNextDayFeature",
  "name": "willFailNextDayFeature",
  "subjectType": "WindTurbine",
  "legacy": {
    "metric": "WillFailNextDay",
    "interval": "HOUR"
  },
  "description": "A binary variable indicating if the wind turbine will fail in the next 24 hours."
}
```

---

Here is a feature set, from the file `seed/Feature.Set/WindTurbine#metricsFeatureSet2.json`:
```
{
  "type": "Feature.Set",
  "id": "WindTurbine#metricsFeatureSet2",
  "name": "metricsFeatureSet2",
  "subjectType": "WindTurbine",
  "description": "Max difference features, created via seed",
  "interval": "HOUR",
  "features": [
    "activePowerDiffFeature",
    "generatorRotationSpeedDiffFeature",
    "gearOilTemperatureDiffFeature"
  ]
}
```

Here is a second feature set, from the file `seed/Feature.Set/WindTurbine#labelFeatureSet.json`:
```
{
  "type": "Feature.Set",
  "id": "WindTurbine#labelFeatureSet",
  "name": "labelFeatureSet",
  "subjectType": "WindTurbine",
  "description": "label feature set, created via seed",
  "interval": "HOUR",
  "features": [
    "willFailNextDayFeature"
  ]
}
```


---
**Note:**

If you want to seed a Feature with a specific materialization time range, you can add that
to the JSON as follows:
```
{
  "type": "Feature",
  ...
  "materializeTimeRange": {
    "type": "Pair<string, string>",
    "fst": "dateTime(\"2012-01-01\")",
    "snd": "dateTime(\"2013-01-01\")"
  }
}  
```
---

Now, we can verify that the seeded features and feature set are present.

In [28]:
c3.Feature.eval(projection='id,name,description', filter="contains(description, 'seed')")

subject  ...                                        description
0             WindTurbine#activePowerDiffFeature  ...      Metric-backed feature, seeded in application.
1      WindTurbine#gearOilTemperatureDiffFeature  ...  Metric-backed feature, seeded in application. ...
2  WindTurbine#generatorRotationSpeedDiffFeature  ...  Metric-backed feature, seeded in application. ...
3             WindTurbine#willFailNextDayFeature  ...  A seeded feature that can be used as a label. ...

[4 rows x 3 columns]

In [29]:
c3.Feature.Set.eval(projection='id,name,description', filter="contains(description, 'seed')")

subject  ...                                        description
0     WindTurbine#labelFeatureSet  ...  Feature is True if a failure should be predict...
1  WindTurbine#metricsFeatureSet2  ...  These are metrics-backed features, created via...

[2 rows x 3 columns]

### Materialize the seeded features
We still need to explicitly materialize our features to see the data. In production, this will be done via a cron job.

In [30]:
for f_id in ['WindTurbine#activePowerDiffFeature',
             'WindTurbine#gearOilTemperatureDiffFeature',
             'WindTurbine#generatorRotationSpeedDiffFeature',
             'WindTurbine#willFailNextDayFeature']:
    print(f"Materializing {f_id}...", end='')
    f = c3.Feature.make({'id':f_id, 'subjectType':c3.WindTurbine}).get()
    f.materialize(sync=True)
    print("successful.")

Materializing WindTurbine#activePowerDiffFeature...successful.
Materializing WindTurbine#gearOilTemperatureDiffFeature...successful.
Materializing WindTurbine#generatorRotationSpeedDiffFeature...successful.
Materializing WindTurbine#willFailNextDayFeature...successful.


### Retrieving the data from the seeded features

In [31]:
projection = 'activePowerDiffFeature, generatorRotationSpeedDiffFeature, gearOilTemperatureDiffFeature, willFailNextDayFeature'
eval_result = c3.WindTurbine.eval(projection=projection, filter=SUBJECT_FILTER, 
                                  start='2022-01-02', end='2022-01-03', interval='HOUR')

eval_result.head()

subject  ... willFailNextDayFeature
0  demo_TURBINE-1  ...                    0.0
1  demo_TURBINE-1  ...                    0.0
2  demo_TURBINE-1  ...                    0.0
3  demo_TURBINE-1  ...                    0.0
4  demo_TURBINE-1  ...                    0.0

[5 rows x 6 columns]

We can also reference our feature sets to retrieve data via `evalFeatureSetBatch`.

In [32]:
fs = c3.Feature.Set(id='WindTurbine#metricsFeatureSet2').get()
fs.materialize(sync=True)
fs_eval_result = c3.WindTurbine.evalFeatureSetBatch(filter=SUBJECT_FILTER, featureSet=fs, 
                                                    start='2022-01-02', end='2022-01-03')

fs_eval_result.head()

subject  ... generatorRotationSpeedDiffFeature
0  demo_TURBINE-1  ...                              56.0
1  demo_TURBINE-1  ...                              56.0
2  demo_TURBINE-1  ...                              56.0
3  demo_TURBINE-1  ...                              56.0
4  demo_TURBINE-1  ...                              56.0

[5 rows x 5 columns]

In [33]:
fs = c3.Feature.Set(id='WindTurbine#labelFeatureSet').get()
fs.materialize(sync=True)
fs_eval_result = c3.WindTurbine.evalFeatureSetBatch(filter=SUBJECT_FILTER, featureSet=fs, 
                                                    start='2022-01-02', end='2022-01-03')

fs_eval_result.head()

subject           timestamp  willFailNextDayFeature
0  demo_TURBINE-1 2022-01-02 00:00:00                     0.0
1  demo_TURBINE-1 2022-01-02 01:00:00                     0.0
2  demo_TURBINE-1 2022-01-02 02:00:00                     0.0
3  demo_TURBINE-1 2022-01-02 03:00:00                     0.0
4  demo_TURBINE-1 2022-01-02 04:00:00                     0.0